In [ ]:
pip install -r requirements.txt

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_........................."

In [ ]:
# Check if the Hugging Face Hub API token is set
import os

api_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

if api_token:
    print(f"Hugging Face Hub API token is set: {api_token}")
else:
    print("Hugging Face Hub API token is not set. Please make sure to set it.")


In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/gdrive/MyDrive/...........Agriculture.csv')
documents = loader.load()


In [ ]:
documents

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=1110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
print(wrap_text_preserve_newlines(str(documents[0])))

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

In [ ]:
docs[0]

#Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [ ]:
# !pip install faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "What was the primary cropping system in Big Island, Hawaii, from 750 to 1400 CE?"
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":1512})

In [ ]:
def get_explanation(query):

    docs = db.similarity_search(query)
    print("Explanation:", docs[0].page_content)

query = "What was the primary cropping system in Big Island, Hawaii, from 750 to 1400 CE?"
get_explanation(query)

In [ ]:
def get_explanation(db):
    query_count = 0

    while True:
        # Get query from user
        query = input("Enter a query (or type 'exit' to exit): ").strip()

        if query.lower() == 'exit':
            break

        # Perform similarity search
        docs = db.similarity_search(query)

        # Display the Explanation from the first result
        explanation = docs[0].page_content
        print(f"\nQuery : {query_count + 1}")
        print("Explanation:", explanation)

        # Increment query count
        query_count += 1

    print("Exiting the loop.")

# Example usage
get_explanation(db)


In [ ]:
query = " Historical Productivity range for Cahokia from 800 to 1500 CE?"
embeddings = HuggingFaceEmbeddings()
answer_with_description = get_answer_with_description(query, db, embeddings)

print(answer_with_description)

In [ ]:
# import pickle

# # Save the db object to a pickle file
# with open('/content/gdrive/MyDrive/bluechiptech.asia/LangChain-Chat-bot/db.pkl', 'wb') as file:
#     pickle.dump(db, file)

# # Confirm that the file is saved
# print("DB object saved to db.pkl")
